# Comp3419 Assignment 2 Option1
## SID: 500177868

### Initial Preparation

In [ ]:
## import packages
import sys
import os
import cv2
import numpy as np
import imutils
import pygame
import random
import PIL.Image as Image
from pygame.locals import *
from random import randint
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
## video path
path_to_origin_video = 'Opt1-MarionetteMovements.mov'
path_to_Final_video = 'SID500177868_Asgmt2Opt1_output.avi'

## Frame dir
Motion_Capture_save_path = './motion/'
Final_save_path = './final/'

## create directory
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
## check whether dir is exist and create if not
create_dir_if_not_exists(Motion_Capture_save_path)
create_dir_if_not_exists(Final_save_path)

### Motion Capture

In [ ]:
def open_video(path):
    ## open video and check whether dir is exist and create if not
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print('{} not opened'.format(path))
        sys.exit(1)
    return cap

In [ ]:
def find_red(frame):
    ## list to store all red markers
    red_markers = []
    ## colour range of red in monkey video
    lower_red = np.array([0,20,150])
    upper_red = np.array([10,255,255])
    mask = cv2.inRange(frame, lower_red, upper_red)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    countours = cv2.findContours(mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    # contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    countours = imutils.grab_contours(countours)
    for c in countours:
        area = cv2.contourArea(c)
        if area > 20: 
            M = cv2.moments(c)
            if M["m00"] > 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                red_markers.append((cx, cy))
    return red_markers

In [ ]:
def adjust_markers(markers, pre):
    ## output order in left leg, right leg, body, leftwing, rightwing, 
    new_markers = []
    
    for h in pre:
        new_markers.append(h)
    min_y = 600
    max_y = 0
    for m in markers:
        if m[1] < min_y:
            min_y = m[1]
        if m[1] > max_y:
            max_y = m[1]
    
    ## category
    bottom = []
    top = []
    center = []
    for i in range(len(markers)):
        if markers[i][1] > (max_y + min_y)/2 + 50:
            bottom.append(markers[i])
        if markers[i][1] < (max_y + min_y)/2 + 50:
            top.append(markers[i])
    
    ## through bottom
    max_x = 0
    min_x = 600
    for m in bottom:
        if m[0] > max_x:
            max_x = m[0]
        if m[0] < min_x:
            min_x = m[0]
    if len(bottom) == 2:
        for m in bottom:
            if m[0] == min_x:
                new_markers[0] = m
            else:
                new_markers[1] = m
    elif len(bottom) == 1:
        if (bottom[0][0] - pre[0][0])+(bottom[0][1] - pre[0][1]) < (bottom[0][0] - pre[1][0])+(bottom[0][1] - pre[1][1]):
            new_markers[0] = bottom[0]
            new_markers[1] = pre[1]
        else:
            new_markers[0] = pre[0]
            new_markers[1] = bottom[0]
    elif len(bottom) > 2:
        left_b = []
        right_b = []
        for b in bottom:
            if b[0] < (max_x + min_x)/2:
                left_b.append(b)
            else:
                right_b.append(b)
        sum_lx = 0
        sum_ly = 0
        for leftb in left_b:
            sum_lx += leftb[0]
            sum_ly += leftb[1]
        new_markers[0] = (int(sum_lx/len(left_b)),int(sum_ly/len(left_b)))
        sum_rx = 0
        sum_ry = 0
        for rightb in right_b:
            sum_rx += rightb[0]
            sum_ry += rightb[1]
        new_markers[1] = (int(sum_rx/len(right_b)),int(sum_ry/len(right_b)))
    
    ## through top
    max_x = 0
    min_x = 600
    for m in top:
        if m[0] > max_x:
            max_x = m[0]
        if m[0] < min_x:
            min_x = m[0]
    
    for t in range(len(top)):
        if top[t][0] > ((max_x + min_x)/2-40) and top[t][0] < ((max_x + min_x)/2+40):
            center.append(top[t])
    
    for c in center:
        top.remove(c)
        
    if len(top) == 2:
        for m in top:
            if m[0] == min_x:
                new_markers[3] = m
            else:
                new_markers[4] = m
    elif len(top) == 1:
        if (top[0][0] - pre[3][0])+(top[0][1] - pre[3][1]) < (top[0][0] - pre[4][0])+(top[0][1] - pre[4][1]):
            new_markers[3] = top[0]
            new_markers[4] = pre[4]
        else:
            new_markers[3] = pre[3]
            new_markers[4] = top[0]
    elif len(top) > 2:
        left_b = []
        right_b = []
        for b in top:
            if b[0] < (max_x + min_x)/2:
                left_b.append(b)
            else:
                right_b.append(b)
        sum_lx = 0
        sum_ly = 0
        for leftb in left_b:
            sum_lx += leftb[0]
            sum_ly += leftb[1]
        new_markers[3] = (int(sum_lx/len(left_b)),int(sum_ly/len(left_b)))
        sum_rx = 0
        sum_ry = 0
        for rightb in right_b:
            sum_rx += rightb[0]
            sum_ry += rightb[1]
        new_markers[4] = (int(sum_rx/len(right_b)),int(sum_ry/len(right_b)))

    ## through center
    if center == []:
        new_markers[2] = pre[2]
    else:
        sum_x = 0
        sum_y = 0
        for m in center:
            sum_x += m[0]
            sum_y += m[1]
        new_markers[2] = (int(sum_x/len(center)),int(sum_y/len(center)))
 
    return new_markers

In [ ]:
## open video
cap = open_video(path_to_origin_video)

# Create a list to store the sequence of captured body motions
body_motions = []

## loop for motion capture
frame_count = 0
pre_marks = [(240, 260), (261, 244), (259, 173), (189, 120), (276, 72)]
while(1):
    ret, frame = cap.read()
    if not ret:
        print('Video Reach End')
        break
    # Convert the frame to the HSV colour
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    if find_red(hsv_frame):
        red_markers = find_red(hsv_frame)
        ## ensure each time have 5 markers
        red_markers = adjust_markers(red_markers, pre_marks)
        pre_marks = red_markers
        for m in red_markers:
            cv2.circle(frame, m, 5, (0, 0, 255), -1)
        ## store motion coordinates
        body_motions.append(red_markers)
    cv2.imwrite(Motion_Capture_save_path + 'composite%d.tif' % frame_count, frame)
    frame_count += 1
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
# Release video capture and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
for i in range(len(body_motions)):
    new_x = 0
    new_y = 0
    new_x = body_motions[i][0][0] + (body_motions[i][2][0] - body_motions[i][0][0])/5
    new_y = body_motions[i][0][1] + (body_motions[i][2][1] - body_motions[i][0][1])/5
    body_motions[i][0] = (int(new_x),int(new_y))
    new_x = 0
    new_y = 0
    new_x = body_motions[i][1][0] + (body_motions[i][2][0] - body_motions[i][1][0])/5
    new_y = body_motions[i][1][1] + (body_motions[i][2][1] - body_motions[i][1][1])/5
    body_motions[i][1] = (int(new_x),int(new_y))
    new_x = 0
    new_y = 0
    new_x = body_motions[i][3][0] + (body_motions[i][2][0] - body_motions[i][3][0])/5
    new_y = body_motions[i][3][1] + (body_motions[i][2][1] - body_motions[i][3][1])/5
    body_motions[i][3] = (int(new_x),int(new_y))
    new_x = 0
    new_y = 0
    new_x = body_motions[i][4][0] + (body_motions[i][2][0] - body_motions[i][4][0])/5
    new_y = body_motions[i][4][1] + (body_motions[i][2][1] - body_motions[i][4][1])/5
    body_motions[i][4] = (int(new_x),int(new_y))
print(body_motions)

### Using Pygame to impete two event

In [ ]:
pygame.init()

screen = pygame.display.set_mode((600, 350))
pygame.display.set_caption('SID500177868_Asgmt2Opt1')

## load all images
bg1 = pygame.image.load("./images/background1.jpg")
bg2 = pygame.image.load("./images/background2.jpg")
otter = pygame.image.load("./images/otter.jpg")
gem = pygame.image.load("./images/gem.jpg")
ink = pygame.image.load("./images/ink.jpg")
body = pygame.image.load("./images/body.jpg")
left = pygame.image.load("./images/left.jpg")
left_wing = pygame.image.load("./images/leftwing.jpg")
right = pygame.image.load("./images/right.jpg")
right_wing = pygame.image.load("./images/rightwing.jpg")
inks = []
inks_dire = []

## set rect for collide
otter_rect = otter.get_rect()
gem_rect = gem.get_rect()
body_rect = body.get_rect()
left_rect = left.get_rect()
left_wing_rect = left_wing.get_rect()
right_rect = right.get_rect()
right_wing_rect = right_wing.get_rect()
inks_rect = []

## set movement for each character
otter_x = 100
otter_y = 250
## random set gem position
gem_x = random.randint(30,550)
gem_y = random.randint(30,320)
gem_x_direction = 1
gem_y_direction = 1

## motion count
count = 0
## gem time cound
start_time = 0

otter_rect.center = (otter_x,otter_y)
gem_rect.center = (gem_x,gem_y)
body_rect.center = (body_motions[count][2])
left_rect.center = (body_motions[count][0])
left_wing_rect.center = (body_motions[count][3])
right_rect.center = (body_motions[count][1])
right_wing_rect.center = (body_motions[count][4])

file_num = 0
## game finish
done = False
dead = 0
## event change
event = 1
clock = pygame.time.Clock()

while not done:
    ## set time
    clock.tick(30)

    if event == 1:
        screen.blit(bg1,(0,0))
    elif event == 2:
        screen.blit(bg2,(0,0))

    ## display sid
    f = pygame.font.SysFont('arial', 12, bold=False, italic=False)
    text = f.render("SID500177868_Asgmt2Opt1",True,(255,0,0))
    textRect =text.get_rect() 
    textRect.center = (80,7)
    screen.blit(text,textRect)

    ## control otter
    for e in pygame.event.get():
        if e.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if e.type == pygame.KEYDOWN:
            if e.key == pygame.K_LEFT or e.key == ord('a'):
                otter_x -= 15
            if e.key == pygame.K_RIGHT or e.key == ord('d'):
                otter_x += 15
            if e.key == pygame.K_UP or e.key == ord('w'):
                otter_y -= 15
            if e.key == pygame.K_DOWN or e.key == ord('s'):
                otter_y += 15
    
    ## if otter collide with boss
    collide_body = pygame.Rect.colliderect(body_rect, otter_rect)
    collide_leftwing = pygame.Rect.colliderect(left_wing_rect, otter_rect)
    collide_righwing = pygame.Rect.colliderect(right_wing_rect, otter_rect)
    collide_left = pygame.Rect.colliderect(left_rect, otter_rect)
    collide_right = pygame.Rect.colliderect(right_rect, otter_rect)
    if collide_body or collide_leftwing or collide_righwing or collide_left or collide_right:
        otter_x = 100
        ottte_y = 330
        # dead += 1
    
    ## in first event otter need cathc gem
    if event == 1:
        ## if otter get gem
        get_gem = pygame.Rect.colliderect(gem_rect, otter_rect)
        if get_gem:
            event = 2
            start_time = pygame.time.get_ticks()
    
        ## random move gem
        # gem_body = pygame.Rect.colliderect(body_rect, gem_rect)
        # gem_leftwing = pygame.Rect.colliderect(left_wing_rect, gem_rect)
        # gem_righwing = pygame.Rect.colliderect(right_wing_rect, gem_rect)
        # gem_left = pygame.Rect.colliderect(left_rect, gem_rect)
        # gem_right = pygame.Rect.colliderect(right_rect, gem_rect)
        if gem_x <= 25 or gem_x >= 575:
        # or gem_body or gem_leftwing or gem_righwing or gem_left or gem_right:
            gem_x_direction *= -1
        if gem_y <= 25 or gem_y >= 325:
            gem_y_direction *= -1
        gem_x += 0.5 * gem_x_direction
        gem_y += 0.5 * gem_y_direction

    ## second event gem will wait for 20 sec
    elif event == 2:
        ## gem stay in corner
        gem_x = 520
        gem_y = 300
        ## display second
        seconds = 20 - int((pygame.time.get_ticks()-start_time)/1000)
        f1 = pygame.font.SysFont('arial', 14, bold=True, italic=False)
        timer = f1.render(str(seconds),True,(255,255,255))
        timerRect =timer.get_rect() 
        timerRect.center = (510,330)
        screen.blit(timer,timerRect)
        ## gem could work
        if seconds == 0:
            done = True
        
        ## avoid ink attack
        ## each second shoot random inks with random direction
        for i in range(0,20):
            if seconds == i:
                inks.append(body_motions[count][2])
                inks_dire.append([random.random(),random.random()])
                inks_rect.append(ink.get_rect())
                break
        for i in range(len(inks)):
            collide_ink = pygame.Rect.colliderect(inks_rect[i], otter_rect)
            if collide_ink:
                otter_x = 100
                ottte_y = 300
                # dead += 1
            new_x = inks[i][0] + inks_dire[i][0] * 8
            new_y = inks[i][1] + inks_dire[i][1] * 8
            inks[i] = (new_x,new_y)
            inks_rect[i] = inks_rect[i].move(inks[i])
            screen.blit(ink,inks[i])

    # if dead == 10:
    #     pygame.main()

    # if pygame.time.get_ticks()/1000 >= 60:
    #     pygame.main()
        
    ## check whether finish
    if done == False:
        otter_rect.center = (otter_x,otter_y)
        gem_rect.center = (gem_x,gem_y)
        body_rect.center = (body_motions[count][2])
        left_rect.center = (body_motions[count][0])
        left_wing_rect.center = (body_motions[count][3])
        right_rect.center = (body_motions[count][1])
        right_wing_rect.center = (body_motions[count][4])

        screen.blit(otter,otter_rect)
        screen.blit(gem,gem_rect)
        ## move follow the motion capture
        screen.blit(body,body_rect)
        screen.blit(left,left_rect)
        screen.blit(left_wing,left_wing_rect)
        screen.blit(right,right_rect)
        screen.blit(right_wing,right_wing_rect)
        ## display red markers in screen
        pygame.draw.circle(screen,(255,0,0),body_motions[count][0],5)
        pygame.draw.circle(screen,(255,0,0),body_motions[count][1],5)
        pygame.draw.circle(screen,(255,0,0),body_motions[count][2],5)
        pygame.draw.circle(screen,(255,0,0),body_motions[count][3],5)
        pygame.draw.circle(screen,(255,0,0),body_motions[count][4],5)
        count += 1
        if count > 900:
            count = 0
    else:
        f2 = pygame.font.SysFont('arial', 20, bold=False, italic=False)
        final = f2.render("the monster BOSS was eliminated"\
                           "the world returned to normal.",True,(255,255,255))
        finalRect =final.get_rect() 
        finalRect.center = (300,175)
        screen.blit(final,finalRect)

    ## Save every frame
    filename = Final_save_path + 'composite%d.tif' % file_num
    pygame.image.save(screen, filename)
    file_num += 1
    
    pygame.display.update()

pygame.quit()

### Output video

In [ ]:

im = Image.open('final/composite0.tif')
img = np.array(im)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cv2.imshow('image', img)
frame_height = img.shape[0]
frame_width = img.shape[1]
out = cv2.VideoWriter(path_to_Final_video, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (int(frame_width), int(frame_height)))
frame_counter = 0
while(1):
    im = Image.open('final/composite%d.tif' % frame_counter)
    img = np.array(im)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img = cv2.imread('final/composite%d.tif' % frame_counter)
    if frame_counter == file_num-1:
        print('No more frames to be loaded')
        break
    out.write(img)
    frame_counter += 1
out.release()
cv2.destroyAllWindows()
